# Autocorrect



<a name='0'></a>
## 0. Overview

You use autocorrect every day on your cell phone and computer. We will explore what really goes on behind the scenes. Of course, the model we are about to implement is not identical to the one used in our phone, but it is still quite good. 

- Get a word count given a corpus
- Get a word probability in the corpus 
- Manipulate strings 
- Filter strings 
- Implement Minimum edit distance to compare strings and to help find the optimal path for the edits. 
- Understand how dynamic programming works


Similar systems are used everywhere. 
- For example, if you type in the word **"I am lerningg"**, chances are very high that you meant to write **"learning"**


# Part 1: Data Preprocessing 

In [3]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [8]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just have to read it in. 
    Output: 
        words: a list containing all the words in the corpus (text file you read) in lower case. 
    """
    words = [] # return this variable correctly

    with open(file_name, encoding='UTF-8') as f:
        contents3 = f.read()

    contents2 = contents3.lower()
    words = re.findall(r'\w+',contents2)
    
    return words

In [9]:

word_l = process_data('sapiens.txt')
vocab = set(word_l)  # this will be new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['kolektif', 'kitap', '63', 'i', 'nceleme', '8', 'hayvanlardan', 'tanrılara', 'sapiens', 'i']
There are 26690 unique words in the vocabulary.


In [10]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}  # fill this with word counts
    word_count_dict = Counter(word_l)

    return word_count_dict

In [15]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('insan',0)}")

There are 26690 key values pairs
The count for the word 'thee' is 326


In [12]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}  # return this variable correctly

    for i in word_count_dict.keys():
        probs[i] = word_count_dict[i] / sum(word_count_dict.values())

    return probs

In [16]:

probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('insan') is {probs['insan']:.4f}")

Length of probs is 26690
P('insan') is 0.0029


# Part 2: String Manipulations

Now, I have computed $P(w_i)$ for all the words in the corpus, you will write a few functions to manipulate strings so that you can edit the erroneous strings and return the right spellings of the words. In this section, I implemented four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


In [17]:

def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_l = []
    split_l = []

    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    
    delete_l = [left+right[1:] for left,right in split_l if right ]

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [18]:
delete_word_l = delete_letter(word="insan",
                        verbose=True)

input word insan, 
split_l = [('', 'insan'), ('i', 'nsan'), ('in', 'san'), ('ins', 'an'), ('insa', 'n')], 
delete_l = ['nsan', 'isan', 'inan', 'insn', 'insa']


In [20]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:])for i in range(len(word))]
    switch_l =[left[:-1] + right[0] + left[-1] + right[1:]  for left,right in split_l if left]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [21]:
switch_word_l = switch_letter(word="insan",
                         verbose=True)

Input word = insan 
split_l = [('', 'insan'), ('i', 'nsan'), ('in', 'san'), ('ins', 'an'), ('insa', 'n')] 
switch_l = ['nisan', 'isnan', 'inasn', 'insna']


In [22]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    ### START CODE HERE ###
    split_l = [(word[:i], word[i:]) for i in range(len(word))] 
    replace_ll = [left + c + right[1:] for left, right in split_l for c in letters if not (left + c + right[1:]==word)]
    replace_set = set(replace_ll)
    ### END CODE HERE ###
    
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [23]:
replace_l = replace_letter(word='insan',
                              verbose=True)

Input word = insan 
split_l = [('', 'insan'), ('i', 'nsan'), ('in', 'san'), ('ins', 'an'), ('insa', 'n')] 
replace_l ['ansan', 'bnsan', 'cnsan', 'dnsan', 'ensan', 'fnsan', 'gnsan', 'hnsan', 'iasan', 'ibsan', 'icsan', 'idsan', 'iesan', 'ifsan', 'igsan', 'ihsan', 'iisan', 'ijsan', 'iksan', 'ilsan', 'imsan', 'inaan', 'inban', 'incan', 'indan', 'inean', 'infan', 'ingan', 'inhan', 'inian', 'injan', 'inkan', 'inlan', 'inman', 'innan', 'inoan', 'inpan', 'inqan', 'inran', 'insaa', 'insab', 'insac', 'insad', 'insae', 'insaf', 'insag', 'insah', 'insai', 'insaj', 'insak', 'insal', 'insam', 'insao', 'insap', 'insaq', 'insar', 'insas', 'insat', 'insau', 'insav', 'insaw', 'insax', 'insay', 'insaz', 'insbn', 'inscn', 'insdn', 'insen', 'insfn', 'insgn', 'inshn', 'insin', 'insjn', 'inskn', 'insln', 'insmn', 'insnn', 'inson', 'inspn', 'insqn', 'insrn', 'inssn', 'instn', 'insun', 'insvn', 'inswn', 'insxn', 'insyn', 'inszn', 'intan', 'inuan', 'invan', 'inwan', 'inxan', 'inyan', 'inzan', 'iosan', 'ipsan', 

In [24]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word) + 1)]
    insert_l = [left + c + right for left,right in split_l for c in letters]
   
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [25]:
insert_l = insert_letter('insan', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word insan 
split_l = [('', 'insan'), ('i', 'nsan'), ('in', 'san'), ('ins', 'an'), ('insa', 'n'), ('insan', '')] 
insert_l = ['ainsan', 'binsan', 'cinsan', 'dinsan', 'einsan', 'finsan', 'ginsan', 'hinsan', 'iinsan', 'jinsan', 'kinsan', 'linsan', 'minsan', 'ninsan', 'oinsan', 'pinsan', 'qinsan', 'rinsan', 'sinsan', 'tinsan', 'uinsan', 'vinsan', 'winsan', 'xinsan', 'yinsan', 'zinsan', 'iansan', 'ibnsan', 'icnsan', 'idnsan', 'iensan', 'ifnsan', 'ignsan', 'ihnsan', 'iinsan', 'ijnsan', 'iknsan', 'ilnsan', 'imnsan', 'innsan', 'ionsan', 'ipnsan', 'iqnsan', 'irnsan', 'isnsan', 'itnsan', 'iunsan', 'ivnsan', 'iwnsan', 'ixnsan', 'iynsan', 'iznsan', 'inasan', 'inbsan', 'incsan', 'indsan', 'inesan', 'infsan', 'ingsan', 'inhsan', 'inisan', 'injsan', 'inksan', 'inlsan', 'inmsan', 'innsan', 'inosan', 'inpsan', 'inqsan', 'inrsan', 'inssan', 'intsan', 'inusan', 'invsan', 'inwsan', 'inxsan', 'inysan', 'inzsan', 'insaan', 'insban', 'inscan', 'insdan', 'insean', 'insfan', 'insgan', 'inshan', 'insian'

In [26]:

def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    
    edit_one_set = set()
    one_list = insert_letter(word) + delete_letter(word) + replace_letter(word)
    if allow_switches:
        edit_one_set = set(switch_letter(word) + one_list)
    else:
        edit_one_set = set(one_list)

    return edit_one_set

In [27]:
tmp_word = "insan"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word insan 
edit_one_l 
['ainsan', 'ansan', 'binsan', 'bnsan', 'cinsan', 'cnsan', 'dinsan', 'dnsan', 'einsan', 'ensan', 'finsan', 'fnsan', 'ginsan', 'gnsan', 'hinsan', 'hnsan', 'iansan', 'iasan', 'ibnsan', 'ibsan', 'icnsan', 'icsan', 'idnsan', 'idsan', 'iensan', 'iesan', 'ifnsan', 'ifsan', 'ignsan', 'igsan', 'ihnsan', 'ihsan', 'iinsan', 'iisan', 'ijnsan', 'ijsan', 'iknsan', 'iksan', 'ilnsan', 'ilsan', 'imnsan', 'imsan', 'inaan', 'inan', 'inasan', 'inasn', 'inban', 'inbsan', 'incan', 'incsan', 'indan', 'indsan', 'inean', 'inesan', 'infan', 'infsan', 'ingan', 'ingsan', 'inhan', 'inhsan', 'inian', 'inisan', 'injan', 'injsan', 'inkan', 'inksan', 'inlan', 'inlsan', 'inman', 'inmsan', 'innan', 'innsan', 'inoan', 'inosan', 'inpan', 'inpsan', 'inqan', 'inqsan', 'inran', 'inrsan', 'insa', 'insaa', 'insaan', 'insab', 'insabn', 'insac', 'insacn', 'insad', 'insadn', 'insae', 'insaen', 'insaf', 'insafn', 'insag', 'insagn', 'insah', 'insahn', 'insai', 'insain', 'insaj', 'insajn', 'insak', 'ins

In [28]:

def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''    
    edit_two_set = set()

    for new_word in edit_one_letter(word):
        edit_two_set = edit_two_set|edit_one_letter(new_word, allow_switches=allow_switches)
   
    return edit_two_set

In [30]:
tmp_edit_two_set = edit_two_letters("insan")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 36859
First 10 strings ['aainsan', 'aansan', 'aasan', 'abinsan', 'abnsan', 'absan', 'acinsan', 'acnsan', 'acsan', 'adinsan']
Last 10 strings ['zwsan', 'zxinsan', 'zxnsan', 'zxsan', 'zyinsan', 'zynsan', 'zysan', 'zzinsan', 'zznsan', 'zzsan']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


In [31]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []

    if word not in vocab:
        edit_one = edit_one_letter(word)
        edit_one_vocab =edit_one.intersection(vocab)
        edit_two = edit_two_letters(word)
        edit_two_vocab = edit_two.intersection(vocab)
        suggestions = list(edit_one_vocab) or list(edit_two_vocab)
        prob_suggestions = [probs[i] for i in suggestions]
        n_best_inx = np.argsort(prob_suggestions)[-3:][::-1]#n olacak
        n_best = [(suggestions[i],prob_suggestions[i]) for i in n_best_inx]
    else:
        n_best = [(word,1)]

    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [33]:
# Testing implementation - feel free to try other words in my word
my_word = 'insson' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

print(f"data type of corrections {type(tmp_corrections)}")

entered word =  insson 
suggestions =  ['insan', 'sasson']
word 0: insan, probability 0.002874
word 1: sasson, probability 0.000026
data type of corrections <class 'list'>


# Part 3: Minimum Edit Distance


In [34]:

def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
   
    m = len(source) 
    n = len(target) 
    
    D = np.zeros((m+1, n+1), dtype=int)    

    for row in range(1,m+1): 
        D[row,0] = D[row-1,0] + 1
        
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + 1
        
    for row in range(1,m+1): 
        
        for col in range(1,n+1):
            
            r_cost = rep_cost          
                
            if source[row-1] == target[col-1]:
               
                r_cost = 0                
            
            D[row,col] = min(D[row-1,col]+del_cost,D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost)
    
    med = D[m,n]
    
    return D, med

In [36]:
source =  'insson'
target = 'insan '
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  i  n  s  a  n   
#  0  1  2  3  4  5  6
i  1  0  1  2  3  4  5
n  2  1  0  1  2  3  4
s  3  2  1  0  1  2  3
s  4  3  2  1  2  3  4
o  5  4  3  2  3  4  5
n  6  5  4  3  4  3  4


In [39]:
source = "inson"
targets = edit_one_letter(source,allow_switches = False)  #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 1: print(source, t, min_edits)

In [40]:
source = "inson"
targets = edit_two_letters(source,allow_switches = False) #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 2 and min_edits != 1: print(source, t, min_edits)

inson inosne 3
inson nisrn 3
inson hsnon 3
inson xnosn 3
inson nistn 3
inson inqno 3
inson iwnosn 3
inson uinosn 3
inson isnkn 3
inson gnsno 3
inson iqosn 3
inson isnor 3
inson insnso 3
inson nision 3
inson isnoy 3
inson nisyn 3
inson izsno 3
inson isfnon 3
inson inocsn 3
inson ysnon 3
inson nisogn 3
inson inmsno 3
inson nisoin 3
inson ssnon 3
inson isdnon 3
inson irnsno 3
inson linsno 3
inson nisonf 3
inson cnsno 3
inson isnod 3
inson niswn 3
inson naison 3
inson isnosn 3
inson niston 3
inson niion 3
inson wisnon 3
inson inorsn 3
inson isnxn 3
inson isnyn 3
inson ihnsno 3
inson ibnosn 3
inson snsno 3
inson insnbo 3
inson insnto 3
inson nisow 3
inson inolsn 3
inson imosn 3
inson isnono 3
inson yisnon 3
inson inose 3
inson isnoqn 3
inson ixnosn 3
inson itosn 3
inson lsnon 3
inson nisoh 3
inson iqnsno 3
inson isnonu 3
inson nisocn 3
inson kisnon 3
inson sisnon 3
inson tnosn 3
inson inrsno 3
inson tinosn 3
inson nnosn 3
inson knosn 3
inson nmison 3
inson inano 3
inson isnopn 3
inson nisno

inson inosr 3
inson inlno 3
inson nisbon 3
inson nisons 3
inson ixosn 3
inson nicon 3
inson nisown 3
inson ingno 3
inson niseon 3
inson qsnon 3
inson insnyo 3
inson gnosn 3
inson nisen 3
inson incno 3
inson nzison 3
inson isnan 3
inson sinsno 3
inson isnoyn 3
inson iynsno 3
inson isnom 3
inson iksno 3
inson inasno 3
inson isnonc 3
inson nisoen 3
inson irnosn 3
inson isnov 3
inson ijsno 3
inson inosnc 3
inson iwnsno 3
inson isnonw 3
inson kinsno 3
inson inusno 3
inson nisoi 3
inson iinosn 3
inson iiosn 3
inson fnsno 3
inson nisoni 3
inson isnofn 3
inson inosj 3
inson iposn 3
inson injsno 3
inson isnodn 3
inson izosn 3
inson inosni 3
inson insnio 3
inson nisxn 3
inson nishn 3
inson nifon 3
inson insnlo 3
inson isnown 3
inson osnon 3
inson nkison 3
inson nfison 3
inson risnon 3
inson inino 3
inson iunosn 3
inson dinosn 3
inson ifnosn 3
inson inosnw 3
inson ishnon 3
inson niszon 3
inson nisov 3
inson insnho 3
inson iynosn 3
inson tinsno 3
inson nisonc 3
inson nisoj 3
inson insnko 3
inson i